# Optimising SQL Queries
© Explore Data Science Academy

## Learning Objectives

In this train, you will learn:
- Common Table Expressions,
- How to optimise SQL queries, and
- How to measure the execution time of a block of code in Jupyter.


## Outline

In this train, we'll go through the following: 
- Why optimise queries?
- Common Table Expressions (CTEs).
- Write efficient SQL queries:
    - Column names instead of *
    - Create JOINS with INNER, not WHERE
    - Avoid using the wildcard (%) at the beginning of a predicate
    - Avoid using functions in predicates
    - Avoid using calculated fields in the JOIN and WHERE clauses

## US Flights Database
In this train, we'll be using the US Flights database with actual US flights data to illustrate the optimisation techniques with large amounts of data. We will also use iPython's built-in `%%time` magic command which will measure the execution time of code within a given jupyter cell. To follow along, you will need to download the `flights.db` database file from Athena. The flights database consist of the following tables:

- **flights**:     all domestic flights in the USA in 2008
- **carriers**:     lookup table for all the carriers
- **airports**:     lookup table for all the airports
- **planes**:     lookup table for the planes

For your convenience, we have also provided the flights database ER diagram:



<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/flights_db_ER.png" width=30% align="left">

Loading SQL magics and load database:

In [1]:
%load_ext sql

# Load SQLite database 
# Make sure this file is in the same directory as your notebook
%sql sqlite:///flights.db 

In [2]:
%%sql
SELECT name
FROM sqlite_master
WHERE type IN ('table', 'view') AND name NOT LIKE 'sqlite_%'
ORDER BY 1

 * sqlite:///flights.db
Done.


name
airports
carriers
flights
planes
sysdiagrams


In [3]:
%%sql
--How many different carriers are there in the database
PRAGMA table_info([carriers])

 * sqlite:///flights.db
Done.


cid,name,type,notnull,dflt_value,pk
0,Code,varchar(10),0,NULL,0
1,Description,varchar(100),0,NULL,0


In [4]:
%%sql
SELECT *
FROM carriers
LIMIT 5

 * sqlite:///flights.db
Done.


Code,Description
02Q,Titan Airways
04Q,Tradewind Aviation
05Q,"Comlux Aviation, AG"
06Q,Master Top Linhas Aereas Ltd.
07Q,Flair Airlines Ltd.


In [5]:
%%sql
SELECT COUNT (DISTINCT Code)
FROM carriers

 * sqlite:///flights.db
Done.


COUNT (DISTINCT Code)
1490


In [6]:
%%sql
SELECT COUNT (Code)
FROM carriers

 * sqlite:///flights.db
Done.


COUNT (Code)
1490


In [7]:
%%sql
SELECT Code, Description 
FROM carriers
WHERE Code IS NULL

 * sqlite:///flights.db
Done.


Code,Description
None,North American Airlines


In [8]:
%%sql
WITH CTE AS (SELECT Code, lower(Description) AS Des
            FROM carriers)
SELECT Code, Des
FROM CTE
WHERE CTE.Des  LIKE "north american airlines%"

 * sqlite:///flights.db
Done.


Code,Des
AMA,north american airlines inc.
None,north american airlines


In [9]:
%%sql
SELECT COUNT (DISTINCT Description)
FROM carriers

 * sqlite:///flights.db
Done.


COUNT (DISTINCT Description)
1491


In [10]:
%%sql
SELECT COUNT ( *)
FROM carriers

 * sqlite:///flights.db
Done.


COUNT ( *)
1491


name
airports
carriers
flights
planes
sysdiagrams


In [11]:
%%sql
--How many different carriers are there in the database
PRAGMA table_info([flights])

 * sqlite:///flights.db
Done.


cid,name,type,notnull,dflt_value,pk
0,index,INTEGER,0,NULL,0
1,Date,longtext,0,None,0
2,DayOfWeek,INTEGER,0,NULL,0
3,DepTime,double,0,NULL,0
4,CRSDepTime,longtext,0,None,0
5,ArrTime,double,0,NULL,0
6,CRSArrTime,longtext,0,None,0
7,UniqueCarrier,longtext,0,None,0
8,FlightNum,longtext,0,None,0
9,TailNum,longtext,0,None,0


In [12]:
%%sql
--How long was the longest delay before departure 
SELECT MAX(DepDelay)
FROM flights


 * sqlite:///flights.db
Done.


MAX(DepDelay)
1355.0


In [13]:
%%sql
SELECT *
FROM flights
LIMIT 5

 * sqlite:///flights.db
Done.


index,Date,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008/1/3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150,116.0,-14.0,8.0,IAD,TPA,810,4,8,0,None,0,None,None,None,None,None
1,2008/1/3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145,113.0,2.0,19.0,IAD,TPA,810,5,10,0,None,0,None,None,None,None,None
2,2008/1/3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90,76.0,14.0,8.0,IND,BWI,515,3,17,0,None,0,None,None,None,None,None
3,2008/1/3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90,78.0,-6.0,-4.0,IND,BWI,515,3,7,0,None,0,None,None,None,None,None
4,2008/1/3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90,77.0,34.0,34.0,IND,BWI,515,3,10,0,None,0,2,0,0,0,32


In [14]:
%%sql
--How many flights departed on the 28th of January 2008
SELECT COUNT( FlightNum)
FROM flights 
WHERE Date = '2008/1/28' AND ActualElapsedTime IS NOT NULL

 * sqlite:///flights.db
Done.


COUNT( FlightNum)
19445


In [15]:
%%sql
--How many flights departed on the 28th of January 2008
SELECT COUNT( FlightNum)
FROM flights 
WHERE Date = '2008/1/28' 

 * sqlite:///flights.db
Done.


COUNT( FlightNum)
20149


In [16]:
%%sql
--How many flights departed on the 28th of January 2008
SELECT COUNT( FlightNum)
FROM flights 
WHERE Date = '2008/1/28' AND Airtime > 0

 * sqlite:///flights.db
Done.


COUNT( FlightNum)
20149


In [17]:
%%sql
--How many flights departed on the 28th of January 2008
SELECT COUNT( FlightNum)
FROM flights 
WHERE Date = '2008/1/28' AND CancellationCode IS NULL

 * sqlite:///flights.db
Done.


COUNT( FlightNum)
19495


In [18]:
%%sql
-- What is the distance between MDW and HOU
SELECT Origin,Dest,Distance
FROM flights
WHERE Origin = 'MDW' AND Dest = 'HOU'
LIMIT 1

 * sqlite:///flights.db
Done.


Origin,Dest,Distance
MDW,HOU,937


In [19]:
%%sql
-- Which day of the week had the highest number of cancelled flights
SELECT DayofWeek, SUM(cancelled) AS cancel
FROM flights
GROUP BY DayofWeek
ORDER BY cancel DESC

 * sqlite:///flights.db
Done.


DayOfWeek,cancel
4,3093
2,2993
3,2645
1,2617
5,2049
6,1984
7,1927


In [20]:
%%sql
--How many Airports have the word 'international' in their name ?
PRAGMA table_info([airports])

 * sqlite:///flights.db
Done.


cid,name,type,notnull,dflt_value,pk
0,iata,varchar(10),0,NULL,0
1,airport,varchar(100),0,NULL,0
2,city,varchar(100),0,NULL,0
3,state,varchar(50),0,NULL,0
4,country,varchar(50),0,NULL,0
5,lat,"float(24,0)",0,NULL,0
6,long,"float(24,0)",0,NULL,0


In [21]:
%%sql
WITH new_arp AS (SELECT lower(airport) AS airpot
                FROM airports)
SELECT COUNT(DISTINCT airpot)
FROM new_arp
WHERE airpot LIKE "%international%"

 * sqlite:///flights.db
Done.


COUNT(DISTINCT airpot)
124


In [22]:
%%sql
--What is the most commonly produced model by the manufacturer "BOEING"
PRAGMA table_info([planes])

 * sqlite:///flights.db
Done.


cid,name,type,notnull,dflt_value,pk
0,tailnum,varchar(10),0,NULL,0
1,type,varchar(100),0,NULL,0
2,manufacturer,varchar(100),0,NULL,0
3,issue_date,varchar(100),0,NULL,0
4,model,varchar(50),0,NULL,0
5,status,varchar(100),0,NULL,0
6,aircraft_type,varchar(100),0,NULL,0
7,engine_type,varchar(100),0,NULL,0
8,year,longtext,0,None,0


In [23]:
%%sql
--What is the most commonly produced model by the manufacturer "BOEING"
SELECT *
FROM planes
LIMIT 5

 * sqlite:///flights.db
Done.


tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
N050AA,None,None,None,None,None,None,None,None
N051AA,None,None,None,None,None,None,None,None
N052AA,None,None,None,None,None,None,None,None
N054AA,None,None,None,None,None,None,None,None
N055AA,None,None,None,None,None,None,None,None


In [24]:
%%sql
--What is the most commonly produced model by the manufacturer "BOEING"
WITH pl AS (
                SELECT upper(manufacturer) AS manfac, model, aircraft_type
                FROM planes
)
SELECT model, COUNT(aircraft_type) AS act
FROM pl
WHERE manfac = 'BOEING'
GROUP BY model
ORDER BY act DESC
LIMIT 3

 * sqlite:///flights.db
Done.


model,act
737-7H4,308
737-3H4,147
757-232,112


In [25]:
%%sql
--What manufacturer had the highest average delay (DepDelay + ArrDelay)
SELECT *
FROM flights
LIMIT 5

 * sqlite:///flights.db
Done.


index,Date,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008/1/3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150,116.0,-14.0,8.0,IAD,TPA,810,4,8,0,None,0,None,None,None,None,None
1,2008/1/3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145,113.0,2.0,19.0,IAD,TPA,810,5,10,0,None,0,None,None,None,None,None
2,2008/1/3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90,76.0,14.0,8.0,IND,BWI,515,3,17,0,None,0,None,None,None,None,None
3,2008/1/3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90,78.0,-6.0,-4.0,IND,BWI,515,3,7,0,None,0,None,None,None,None,None
4,2008/1/3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90,77.0,34.0,34.0,IND,BWI,515,3,10,0,None,0,2,0,0,0,32


In [26]:
%%sql
--What manufacturer had the highest average delay (DepDelay + ArrDelay)
SELECT *
FROM planes
LIMIT 5

 * sqlite:///flights.db
Done.


tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
N050AA,None,None,None,None,None,None,None,None
N051AA,None,None,None,None,None,None,None,None
N052AA,None,None,None,None,None,None,None,None
N054AA,None,None,None,None,None,None,None,None
N055AA,None,None,None,None,None,None,None,None


In [27]:
%%sql
WITH comb AS (SELECT f.TailNum AS tail, f.ArrDelay AS arr, f.DepDelay AS dep, p.manufacturer AS man
                FROM flights AS f
                INNER JOIN planes AS p
                ON f.TailNum = p.tailnum)
SELECT man, AVG(arr + dep) AS ans
FROM comb
GROUP BY man
ORDER BY ans DESC
LIMIT 3

 * sqlite:///flights.db
Done.


man,ans
AVIAT AIRCRAFT INC,46.18867924528302
GULFSTREAM AEROSPACE,36.96
SIKORSKY,36.56944444444444


In [28]:
%%sql
--How many planes landed at Los Angeles International Airport 
PRAGMA table_info([airports])

 * sqlite:///flights.db
Done.


cid,name,type,notnull,dflt_value,pk
0,iata,varchar(10),0,NULL,0
1,airport,varchar(100),0,NULL,0
2,city,varchar(100),0,NULL,0
3,state,varchar(50),0,NULL,0
4,country,varchar(50),0,NULL,0
5,lat,"float(24,0)",0,NULL,0
6,long,"float(24,0)",0,NULL,0


In [43]:
%%sql
--How many planes landed at Los Angeles International Airport 
SELECT *
FROM airports
WHERE airport LIKE 'Los Angeles%'

 * sqlite:///flights.db
Done.


iata,airport,city,state,country,lat,long
LAX,Los Angeles International,Los Angeles,CA,USA,34.0,-118.0


In [49]:
%%sql
--How many planes landed at Los Angeles International Airport 
SELECT COUNT(TailNum)
FROM flights
WHERE DEST = 'LAX' AND Cancelled = 0 AND Diverted = 0

 * sqlite:///flights.db
Done.


COUNT(TailNum)
18489


## Why optimise queries?

Depending on the size of the database and the nature of the query, SQL queries can take a long time to run. Also, writing SQL massive queries can potentially clog a server (in a server-based setup). For example, if a server spends too long on a single query then other queries need to "wait in line" for the server to finish processing the one query. As such, it is important that we write more efficient queries. For example, queries that read as much data as is necessary.

## Common Table Expressions (CTEs)

A CTE is a temporary result (i.e. not saved in the database) set that you can reference within a larger query. A CTE is useful in cases where we need to write nested queries and complex joins. CTEs can be recursive or non-recursive and can be better than standard nested queries when:

- The result of the outer query is referenced more than once.
- Writing recursive queries.

### Non-recursive CTE syntax

A non-recursive CTE can be created as follows:

```SQL
WITH CTE AS
(
SELECT Column1, Column2
FROM table1
)
```
and used as follows:

```SQL
SELECT Column1 
FROM CTE
```

**Note:** For clarity of display, we've cleaned the output of all queries. Please re-run these locally or on Google Colab to see their full output. 

## Column names instead of *
**Scenario:** We are only interested in the tail number and delay before departure for each flight. 
 
### Inefficient query

In [30]:
%%time
%%sql

SELECT * 
FROM flights
LIMIT 25

 * sqlite:///flights.db
Done.
CPU times: user 1.87 ms, sys: 959 µs, total: 2.83 ms
Wall time: 2.08 ms


index,Date,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008/1/3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150,116.0,-14.0,8.0,IAD,TPA,810,4,8,0,None,0,None,None,None,None,None
1,2008/1/3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145,113.0,2.0,19.0,IAD,TPA,810,5,10,0,None,0,None,None,None,None,None
2,2008/1/3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90,76.0,14.0,8.0,IND,BWI,515,3,17,0,None,0,None,None,None,None,None
3,2008/1/3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90,78.0,-6.0,-4.0,IND,BWI,515,3,7,0,None,0,None,None,None,None,None
4,2008/1/3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90,77.0,34.0,34.0,IND,BWI,515,3,10,0,None,0,2,0,0,0,32
5,2008/1/3,4,1940.0,1915,2121.0,2110,WN,378,N726SW,101.0,115,87.0,11.0,25.0,IND,JAX,688,4,10,0,None,0,None,None,None,None,None
6,2008/1/3,4,1937.0,1830,2037.0,1940,WN,509,N763SW,240.0,250,230.0,57.0,67.0,IND,LAS,1591,3,7,0,None,0,10,0,0,0,47
7,2008/1/3,4,1039.0,1040,1132.0,1150,WN,535,N428WN,233.0,250,219.0,-18.0,-1.0,IND,LAS,1591,7,7,0,None,0,None,None,None,None,None
8,2008/1/3,4,617.0,615,652.0,650,WN,11,N689SW,95.0,95,70.0,2.0,2.0,IND,MCI,451,6,19,0,None,0,None,None,None,None,None
9,2008/1/3,4,1620.0,1620,1639.0,1655,WN,810,N648SW,79.0,95,70.0,-16.0,0.0,IND,MCI,451,3,6,0,None,0,None,None,None,None,None


### Efficient query

In [31]:
%%time
%%sql

SELECT TailNum, DepDelay 
FROM flights 
LIMIT 25

 * sqlite:///flights.db
Done.
CPU times: user 2.18 ms, sys: 1.41 ms, total: 3.59 ms
Wall time: 2.56 ms


TailNum,DepDelay
N712SW,8.0
N772SW,19.0
N428WN,8.0
N612SW,-4.0
N464WN,34.0
N726SW,25.0
N763SW,67.0
N428WN,-1.0
N689SW,2.0
N648SW,0.0


### Why it’s more efficient
Many SQL developers use SELECT * FROM table to get look at column data which becomes taxing if there are many records (normally a few million or more).

Returning only the required columns reduces the total data being read and subsequently runtime. When using * (i.e. want all columns), it’s important to then limit the number of rows using `LIMIT X`


## Create JOINS with INNER, not WHERE
**Scenario:** We want the carrier description to be added to the flights table.
 
### Inefficient query

In [32]:
%%time
%%sql

SELECT FlightNum, TailNum, Description
FROM flights, carriers
WHERE carriers.Code = flights.UniqueCarrier
LIMIT 25

 * sqlite:///flights.db
Done.
CPU times: user 3.23 ms, sys: 1.28 ms, total: 4.51 ms
Wall time: 2.89 ms


FlightNum,TailNum,Description
335,N712SW,Southwest Airlines Co.
3231,N772SW,Southwest Airlines Co.
448,N428WN,Southwest Airlines Co.
1746,N612SW,Southwest Airlines Co.
3920,N464WN,Southwest Airlines Co.
378,N726SW,Southwest Airlines Co.
509,N763SW,Southwest Airlines Co.
535,N428WN,Southwest Airlines Co.
11,N689SW,Southwest Airlines Co.
810,N648SW,Southwest Airlines Co.


### Efficient query

In [33]:
%%time
%%sql

SELECT FlightNum, TailNum, Description
FROM flights
INNER JOIN carriers
ON carriers.Code = flights.UniqueCarrier
LIMIT 25

 * sqlite:///flights.db
Done.
CPU times: user 3.15 ms, sys: 1.13 ms, total: 4.28 ms
Wall time: 2.84 ms


FlightNum,TailNum,Description
335,N712SW,Southwest Airlines Co.
3231,N772SW,Southwest Airlines Co.
448,N428WN,Southwest Airlines Co.
1746,N612SW,Southwest Airlines Co.
3920,N464WN,Southwest Airlines Co.
378,N726SW,Southwest Airlines Co.
509,N763SW,Southwest Airlines Co.
535,N428WN,Southwest Airlines Co.
11,N689SW,Southwest Airlines Co.
810,N648SW,Southwest Airlines Co.


### Why it’s more efficient
Joining tables using the WHERE clause is similar to doing a CROSS join. This type of join will first generate all possible combinations of the values in the joining columns and then filter these results according to the specified condition, this means we iterate through the combination of records from both tables. However, when a join operation is performed, only records in the left table are iterated through to find matches in right table. 

## Avoid using the wildcard (%) at the beginning of a predicate
**Scenario:** We want the tail number and distance of all flights whose tail number starts with ‘N7’ 
 
### Inefficient query

In [34]:
%%time
%%sql

SELECT TailNum, Distance
FROM flights
WHERE TailNum LIKE '%N7%'
LIMIT 25

 * sqlite:///flights.db
Done.
CPU times: user 2.69 ms, sys: 1.98 ms, total: 4.67 ms
Wall time: 3.29 ms


TailNum,Distance
N712SW,810
N772SW,810
N726SW,688
N763SW,1591
N765SW,162
N778SW,1489
N724SW,220
N786SW,220
N714CB,220
N712SW,2283


### Efficient query

In [35]:
%%time
%%sql

SELECT TailNum, Distance
FROM flights
WHERE TailNum LIKE 'N7%'
LIMIT 25

 * sqlite:///flights.db
Done.
CPU times: user 2.74 ms, sys: 1.71 ms, total: 4.45 ms
Wall time: 3.15 ms


TailNum,Distance
N712SW,810
N772SW,810
N726SW,688
N763SW,1591
N765SW,162
N778SW,1489
N724SW,220
N786SW,220
N714CB,220
N712SW,2283


### Why it’s more efficient
Using a wildcard at the beginning of a pattern will cause a full table scan. If you need to search for text at the beginning of a field, only use a trailing wildcard. This will take advantage of any indexing that is in place.

## Avoid using functions in predicates
**Scenario:** We want to return the origin and departure delay of all flights from a specific origin. 
 
### Inefficient query

In [36]:
%%time
%%sql

SELECT Origin, DepDelay 
FROM flights
WHERE LOWER(Origin) = 'las'
LIMIT 70

 * sqlite:///flights.db
Done.
CPU times: user 2.28 ms, sys: 1.32 ms, total: 3.6 ms
Wall time: 2.5 ms


Origin,DepDelay
LAS,53.0
LAS,-2.0
LAS,83.0
LAS,57.0
LAS,-1.0
LAS,14.0
LAS,2.0
LAS,97.0
LAS,34.0
LAS,16.0


### Efficient query

In [37]:
%%time
%%sql

WITH CTE AS
(SELECT LOWER(Origin) Origin_lower, DepDelay FROM flights)

SELECT * FROM CTE WHERE Origin_lower = 'las'
LIMIT 70

 * sqlite:///flights.db
Done.
CPU times: user 2.41 ms, sys: 1.32 ms, total: 3.74 ms
Wall time: 2.6 ms


Origin_lower,DepDelay
las,53.0
las,-2.0
las,83.0
las,57.0
las,-1.0
las,14.0
las,2.0
las,97.0
las,34.0
las,16.0


### Why it’s more efficient
If we use functions in predicates, the index-based optimisations can’t be used. One alternative method is to use CTE’s instead.

## Avoid using calculated fields in the JOIN and WHERE clauses

**Scenario:** We want to return the flights where the departure delay was more than 20% of the flight time. 
 
### Inefficient query

In [38]:
%%time
%%sql

SELECT TailNum, DepDelay
FROM flights
WHERE AirTime/5 < DepDelay
LIMIT 50

 * sqlite:///flights.db
Done.
CPU times: user 2.54 ms, sys: 1.32 ms, total: 3.86 ms
Wall time: 2.78 ms


TailNum,DepDelay
N464WN,34.0
N726SW,25.0
N763SW,67.0
N334SW,94.0
N263WN,9.0
N674AA,28.0
N643SW,51.0
N724SW,32.0
N786SW,20.0
N394SW,25.0


### Efficient query

In [39]:
%%time
%%sql

WITH CTE AS 
(SELECT AirTime/5 as fifth_time, DepDelay, TailNum FROM flights)

SELECT TailNum, DepDelay 
FROM CTE
WHERE CTE.fifth_time < CTE.DepDelay
LIMIT 50

 * sqlite:///flights.db
Done.
CPU times: user 2.98 ms, sys: 1.63 ms, total: 4.62 ms
Wall time: 3.34 ms


TailNum,DepDelay
N464WN,34.0
N726SW,25.0
N763SW,67.0
N334SW,94.0
N263WN,9.0
N674AA,28.0
N643SW,51.0
N724SW,32.0
N786SW,20.0
N394SW,25.0


### Why it’s more efficient

Calculated fields in JOIN and WHERE clauses also prevent indices from being used. CTEs can be used instead here as well.


## Conclusion

In this train, we covered different strategies for optimising SQL queries. These can be summarised as follows:

- **Column names instead of ***
    - Taxing to return records of all columns for big tables
    - Limit amount of data being read by specifying subset of columns
    
    
- **Create JOINS with INNER, not WHERE**
    - WHERE clause does a CROSS join which considers the combination of rows from both tables
    - Join statements only iterate through records in the left table when looking for matches.
 
 
- **Avoid using the wildcard (%) at the beginning of a predicate**
    - Execution will be slower if % at beginning due to full table scan
    - Use only trailing wildcard when searching for start of word


- **Avoid using functions in predicates**
    - Execution will be slower, due to not using indices
    - Create CTE to handle it instead
     
    
- **Avoid using calculated fields in the JOIN and WHERE clauses**
    - Avoid Calculated Fields in JOIN and WHERE clause
    - Create CTE to handle it instead
    
As demonstrated in the train, another quick way to make queries more efficient, particularly in cases where we don't want the full output of a query, is to use the `LIMIT` keyword. 

## Appendix

The following links may be useful for further learning around the content we've covered: 

 - [SQL Server CTEs](https://www.sqlservertutorial.net/sql-server-basics/sql-server-cte/)
 - [8 ways to fine tune SQL queries](https://www.sisense.com/blog/8-ways-fine-tune-sql-queries-production-databases/)